### 랭체인 도구
- OpenAi 의 function calling 과 tools 기능과 유사
- @tool 데코레이터를 사용하여 함수를 도구로 변환 -> 도구 등록 -> 언어 모델이 함수 호출하여 응답 생성

In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4.1-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='응, 잘 지냈어! 너는 어떻게 지냈어?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 12, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CaEJjg0K0u46YnksP9B3FHC8S4SVo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--1e00db15-6911-4f78-be2f-f6ca93fa2c87-0', usage_metadata={'input_tokens': 12, 'output_tokens': 14, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # (1)tool 데코레이터를 사용하여 함수를 도구로 등록. docstring 또는 descriotion 속성 필수
def get_current_time(timezone : str, location: str)-> str:
  """ 현재 시각을 반환하는 함수

  Args:
    timezone(str) : 타임존 (예 : 'Asia/Seoul) 실제 존재하는 타임존이어야 함
    location (str): 지역명, llm이 전달해주는 지역임 확인.  
  """
  tz = pytz.timezone(timezone)
  now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
  location_and_local_time = f'{timezone} ({location}) 현재시각 {now}' # 타임존, 지역명, 현재시각을 문자열로 반환
  print(location_and_local_time)
  return location_and_local_time

In [3]:
# (2)도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time" : get_current_time}

# (3)도구를 모델에 바인딩 : 모델에 도구를 바인딩하면, 도구를 사용하여 llm 다변을 생성할 수 있음.
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
  SystemMessage(content="너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
   HumanMessage(content='도쿄는 지금 몇시야?'), # response 에 tool_calls 항목 있음.
  #  HumanMessage(content='도쿄는 지금 몇시야?'), # response 에 tool_calls 항목 빈 리스트.
]

# (5) llm_with_tools를 사영하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 지금까지의 messages 확인
for i, msg in enumerate(messages):
  print(f'messages[{i}] : {msg}')

messages[0] : content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.' additional_kwargs={} response_metadata={}
messages[1] : content='도쿄는 지금 몇시야?' additional_kwargs={} response_metadata={}
messages[2] : content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 123, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CaEJkqEG17ZvVZ5UkS1BJBI5aWn0t', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--646f3da4-1253-4635-938a-afcb9b762c79-0' tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Tokyo', 'location': 'Tokyo'}, 'id': 'call_GtNhyrDtEhWVLXbemxhCHy1x', 'type': 'tool_call'}] usag

In [5]:
print(response.tool_calls)  # message[2].tool_calls[0] 과 같음

[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Tokyo', 'location': 'Tokyo'}, 'id': 'call_GtNhyrDtEhWVLXbemxhCHy1x', 'type': 'tool_call'}]


In [6]:
print(response.model_dump_json(indent=2))

{
  "content": "",
  "additional_kwargs": {
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 22,
      "prompt_tokens": 123,
      "total_tokens": 145,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_provider": "openai",
    "model_name": "gpt-4.1-mini-2025-04-14",
    "system_fingerprint": "fp_4c2851f862",
    "id": "chatcmpl-CaEJkqEG17ZvVZ5UkS1BJBI5aWn0t",
    "service_tier": "default",
    "finish_reason": "tool_calls",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "lc_run--646f3da4-1253-4635-938a-afcb9b762c79-0",
  "tool_calls": [
    {
      "name": "get_current_time",
      "args": {
        "timezone": "Asia/Tokyo",
        "location": "Tokyo"
      },
      "id": "

In [7]:
# (7) response.tool_calls 있으면 저장된 함수 호출 정보를 하나씩 가져와 실행하고 결과는 ToolMessage 타입으로
# messages 에 추가
from langchain.messages import ToolMessage
for tool_call in response.tool_calls:
  # 중요 : tool_call 구조 분해
  tool_name = tool_call.get("name") # tool_call["name"]
  tool_args = tool_call.get("args", {})
  tool_call_id = tool_call.get("id") # 이 ID가 매칭 키
  # tool_dict를 사용하여 도구 함수객체 리턴
  selected_tool = tool_dict[tool_name] # tool_dict 를 사용하여 도구 함수객체 리턴
  tool_result = selected_tool.invoke(tool_args)
  print(f'log args -> {tool_call["args"]}') # 도구 호출 시 전달된 인자 출력
  tool_result = selected_tool.invoke(tool_call)
  tool_message = ToolMessage(
    tool_call_id = tool_call_id, # <- 도구 호출 ID와 매칭
    name = tool_name, # <- 도구 이름
    content=str(tool_result) # <- 도구 실행 결과 (문자열)
  )
  messages.append(tool_message)
messages

'''
[
  {
  "name": "get_current_time",
    "args": {
        "timezone": "Asia/Tokyo",
        "location": "Tokyo"
  },
  {
   "id": "call_a6Gkdjbh5k5NBYKSqXHxEg6L",
    "type": "tool_call"
  }
  -> tool_call L 이 값으로 그래도 실행하려면 invoke() 메소드 사용하기
]
'''

Asia/Tokyo (Tokyo) 현재시각 2025-11-10 14:01:30
log args -> {'timezone': 'Asia/Tokyo', 'location': 'Tokyo'}
Asia/Tokyo (Tokyo) 현재시각 2025-11-10 14:01:30


'\n[\n  {\n  "name": "get_current_time",\n    "args": {\n        "timezone": "Asia/Tokyo",\n        "location": "Tokyo"\n  },\n  {\n   "id": "call_a6Gkdjbh5k5NBYKSqXHxEg6L",\n    "type": "tool_call"\n  }\n  -> tool_call L 이 값으로 그래도 실행하려면 invoke() 메소드 사용하기\n]\n'

In [8]:
# (8) 최종 답변
llm_with_tools.invoke(messages)

AIMessage(content='도쿄는 지금 2025년 11월 10일 오후 2시 1분 30초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 207, 'total_tokens': 235, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CaEJmnvmpUM4A1YUTPO6fxftKzjQa', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--0ad1c800-d800-44e3-a4cf-ad4a7ff5037a-0', usage_metadata={'input_tokens': 207, 'output_tokens': 28, 'total_tokens': 235, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
messages[-1].content # -> llm 에 전달해서 내용을 만들어 줘야 합니다.

"content='Asia/Tokyo (Tokyo) 현재시각 2025-11-10 14:01:30' name='get_current_time' tool_call_id='call_GtNhyrDtEhWVLXbemxhCHy1x'"

In [12]:
response = llm_with_tools.invoke(messages)
print(f'AI : {response.content}')

AI : 도쿄는 지금 2025년 11월 10일 14시 01분입니다.
